In [1]:
# Import necessary packages
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import scipy as scp
import matplotlib.pyplot as plt
import os
import dnnregressor_predictor as dnn_pred
import dnnregressor_model_and_input_fn as dnn_model_input
import make_data_wfpt as mdw
import seaborn as sns

In [90]:
# Tukey's five number summary
def tukey_fivenum(vec = [0, 1, 2, 3, 4, 5]):
    tukey = [np.min(vec),
          np.percentile(vec, 25, interpolation='midpoint'),
          np.median(vec),
          np.percentile(vec, 75, interpolation='midpoint'),
          np.max(vec)]
    for v in tukey:
     print(v)

In [2]:
# Define Model path:
model_path  = os.getcwd() + '/keras_models/dnnregressor_choice_rt_09_12_18_18_47_03/model_0'
ckpt_path = os.getcwd() + '/keras_models/dnnregressor_choice_rt_09_12_18_18_47_03/ckpt_0_' + 'final'

# Load model
model = keras.models.load_model(model_path)

In [3]:
# Load weights
model.load_weights(ckpt_path)

In [6]:
# Read relevant dataset and training indices
train_indices = np.loadtxt(os.getcwd() + '/data_storage/train_indices_11000000_from_simulation_mix_09_12_18_18_20_50.csv')
data = pd.read_csv('/data_storage/data_11000000_from_simulation_mix_09_12_18_18_20_50.csv')

datafile used to read in training data: data_storage/train_data_11000000_from_simulation_mix_09_12_18_18_20_50.csv
datafile used to read in test data: data_storage/test_data_11000000_from_simulation_mix_09_12_18_18_20_50.csv


In [72]:
# Split into training and test set according to training indices used for training the neural nets
training_data = data.loc[np.where(train_indices == 0)[0]]
training_data = training_data[['v','a','w','rt', 'choice', 'nf_likelihood']]
test_data = data.loc[np.where(train_indices == 1)[0]]
test_data = test_data[['v','a','w','rt','choice', 'nf_likelihood']]

training_features, _, __, ___ = mdw.train_test_split_rt_choice(
                                                               data = training_data,
                                                               p_train = 1,
                                                               backend = 'keras',
                                                               from_file = False,
                                                               write_to_file = False
                                                               )

test_features, _, __, ___ = mdw.train_test_split_rt_choice(
                                                           data = test_data,
                                                           p_train = 1,
                                                           backend = 'keras',
                                                           from_file = False,
                                                           write_to_file = False
                                                           )

In [73]:
# Predict training data
training_predictions = model.predict(training_features)

# Predict test data
test_predictions = model.predict(test_features)

In [81]:
# Add predictions to dataset
training_data['pred_likelihood'] = training_predictions
training_data['error'] = training_data['nf_likelihood'] - training_data['pred_likelihood']
test_data['pred_likelihood'] = test_predictions
test_data['error'] = test_data['nf_likelihood'] - test_data['pred_likelihood']

In [82]:
# Prepare for plotting
training_data.sort_values('error', axis = 0, ascending = True, inplace = True, kind = 'quicksort', na_position = 'last')
test_data.sort_values('error', axis = 0, ascending = True, inplace = True, kind = 'quicksort', na_position = 'last')

# dataset_long = dataset.melt(id_vars = ['rt', 'choice', 'v'], 
#                             value_vars = ['nf_likelihood', 'pred_likelihood'], 
#                             var_name = 'group', 
#                             value_name = 'likelihood')

In [91]:
# Summary of test data errors
tukey_fivenum((np.sqrt(np.power(test_data['error'], 2))))

9.041716850788362e-07
0.13479458554090662
0.22112407083984964
0.5362792750184986
1049.8618740522465


In [92]:
# Summary of training data errors
tukey_fivenum((np.sqrt(np.power(training_data['error'], 2))))

8.421866892760477e-08
0.6679046681945008
2.325024923367318
11.164078878336213
11942.944229444605


In [93]:
test_data

,v,a,w,rt,choice,nf_likelihood,pred_likelihood,error
1962614,0.045012,0.395324,0.111698,-0.000048,1.0,0.000000e+00,151.814102,-151.814102
8785135,0.640819,1.576764,0.986451,-0.000290,-1.0,0.000000e+00,148.625839,-148.625839
7852837,0.056825,0.130042,0.160175,-0.000437,1.0,0.000000e+00,148.262756,-148.262756
3334328,-1.297910,0.121583,0.267487,-0.000499,1.0,0.000000e+00,122.447853,-122.447853
4734641,0.011242,0.123930,0.100760,0.000002,1.0,2.200823e-12,112.550072,-112.550072
7736199,2.536117,0.175484,0.178814,-0.000959,1.0,0.000000e+00,109.053032,-109.053032
10124386,2.817751,2.869514,0.004789,-0.001197,1.0,0.000000e+00,95.120781,-95.120781
1479789,-1.044043,0.372471,0.057134,-0.000490,1.0,0.000000e+00,92.651100,-92.651100
4567621,-2.340316,1.928877,0.985843,-0.000451,-1.0,0.000000e+00,84.577827,-84.577827
2858046,-1.757979,0.161931,0.971109,-0.000068,-1.0,0.000000e+00,74.480263,-74.480263


In [94]:
training_data

,v,a,w,rt,choice,nf_likelihood,pred_likelihood,error
6577372,1.408578,2.326794,0.155240,6.670812,-1.0,1.324690e-05,268.727081,-268.727068
2155230,-0.254604,0.604608,0.979584,0.823782,-1.0,7.913818e-06,268.401825,-268.401817
1723350,-2.069599,2.919836,0.640076,-1.279078,1.0,0.000000e+00,268.325287,-268.325287
3926193,-0.132746,0.438935,0.488936,0.507682,1.0,3.759703e-05,268.084167,-268.084130
5995206,-1.005699,2.670314,0.449412,11.658762,1.0,1.252910e-06,268.064178,-268.064177
8410053,-2.714583,1.841973,0.618580,12.795518,1.0,5.269435e-28,267.838531,-267.838531
10511382,-1.369422,1.209400,0.818617,-1.140080,-1.0,0.000000e+00,267.736206,-267.736206
8751647,0.387793,1.059985,0.383652,0.278000,1.0,6.677875e-01,268.095154,-267.427366
957577,1.128421,1.020866,0.747363,0.118000,1.0,2.451191e-01,267.669861,-267.424742
2128090,2.630018,1.512728,0.331805,0.225000,-1.0,2.551619e+00,269.830170,-267.278551


In [5]:
np.sign(-1)

-1

In [2]:
import numpy as np


In [6]:
np.unique([-1, 1])

array([-1,  1])